<a href="https://colab.research.google.com/github/akstud/Ocean-object_classification/blob/master/AUV_FP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***IMPORTING SOME BASIC LIBRARIES***

In [ ]:
import tensorflow as tf
import os


***UPLOADING MY DATASET***

It is uploaded in rar format and has been unrared

In [ ]:
#trying to unrar my folder
!unrar x "/content/drive/My Drive/dataset_AUV/train.rar" "/content/drive/My Drive/Colab Notebooks/"

In [ ]:
!unrar x "/content/drive/My Drive/dataset_AUV/test_col.rar" "/content/drive/My Drive/Colab Notebooks/"

***CLASSES THAT I HAVE USED***

In [ ]:
dat=["coral reef","diver","fish","jelly fish","plant","robot","ruins","shark","starfish","turtle"]

***CALLBACKS***

To stop the training at 99% accuracy

In [ ]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if (logs.get('acc') is not None and logs.get('acc') > 0.99):
      print("\nReached 99% accuracy so cancelling training!")
      self.model.stop_training = True
 
callbacks = myCallback()

**MY MODEL**

In [ ]:
model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=( 150,150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
   
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        tf.keras.layers.Conv2D(32, (3,3), activation='relu'),
        
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),

        tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        

        #tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
        #tf.keras.layers.MaxPooling2D(2,2),
        
        tf.keras.layers.Flatten(),
        
        tf.keras.layers.Dense(128,activation = 'relu'),
        
        tf.keras.layers.Dense(10,activation = 'softmax')
    ])

In [ ]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_19 (Conv2D)           (None, 148, 148, 16)      448       
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 74, 74, 16)        0         
_________________________________________________________________
conv2d_20 (Conv2D)           (None, 72, 72, 32)        4640      
_________________________________________________________________
conv2d_21 (Conv2D)           (None, 70, 70, 32)        9248      
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 68, 68, 64)        18496     
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 34, 34, 64)        0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 32, 32, 64)       

***CONFIGURATIONS***
with some chosen hyperparameters

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model.compile(loss = 'categorical_crossentropy',
                  optimizer = RMSprop(lr = 0.001),
                  metrics = ['acc'])

***DATA PREPROCESSING***

my training images are labelled, shuffled, resized in this function

batch size of 64 chosen

and class mode is

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255)

train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Colab Notebooks/train/',
                                                        target_size = (150,150),
                                                        shuffle = True,
                                                        batch_size = 64,
                                                        class_mode = 'categorical')


Found 4109 images belonging to 10 classes.


***TRAINING MY MODEL***

In [ ]:
history = model.fit_generator(train_generator,
                              steps_per_epoch = 8,
                              epochs = 100,
                              verbose = 1,
                              callbacks = [callbacks])

Epoch 1/100
8/8 [==============================] - 3s 367ms/step - loss: 0.7066 - acc: 0.8418
Epoch 2/100
8/8 [==============================] - 3s 371ms/step - loss: 0.1241 - acc: 0.9590
Epoch 3/100
8/8 [==============================] - 3s 313ms/step - loss: 0.1206 - acc: 0.9629
Epoch 4/100
8/8 [==============================] - 3s 362ms/step - loss: 0.2361 - acc: 0.9284
Epoch 5/100
8/8 [==============================] - 3s 398ms/step - loss: 0.1207 - acc: 0.9629
Epoch 6/100
8/8 [==============================] - 2s 310ms/step - loss: 0.1569 - acc: 0.9566
Epoch 7/100
8/8 [==============================] - 2s 307ms/step - loss: 0.1692 - acc: 0.9395
Epoch 8/100
8/8 [==============================] - 3s 317ms/step - loss: 0.0825 - acc: 0.9707
Epoch 9/100
8/8 [==============================] - 3s 352ms/step - loss: 0.1995 - acc: 0.9395
Epoch 10/100
8/8 [==============================] - 3s 356ms/step - loss: 0.2392 - acc: 0.9297
Epoch 11/100
8/8 [==============================] - 3s 431m



NOW I ALSO NEED TO ADD DATA AUGMENTATION TO INCREASE THE NUMBER OF IMAGES IN MY DATASET

***TESTING***

#*** testing the model***

In [ ]:
import numpy as np

from google.colab import files
from keras.preprocessing import image

uploaded=files.upload()
a=0
c=0
for fn in uploaded.keys():
  c=c+1 
  # predicting images
  path='/content/' + fn
  img=image.load_img(path, target_size=(150, 150))
  
  x=image.img_to_array(img)
  x=np.expand_dims(x, axis=0)
  images = np.vstack([x])
  
  classes = model.predict(images, batch_size=10)
  
  if classes[0][0]==1:
    print("coral reef")    
  elif classes[0][1]==1:
    print("divers")
    
  elif classes[0][2]==1:
    print("fish")
    
  elif classes[0][3]==1:
    print("jellyfish")
    
  elif classes[0][4]==1:
    print("plant")
    
  elif classes[0][5]==1:
    print("robot")
    
  elif classes[0][6]==1:
    print("ruins")
    
  elif classes[0][7]==1:
    print("shark")
   
  elif classes[0][8]==1:
    print("starfish")
    a=a+1
  else:
    print("turtle")
    a=a+1
  
print(a*100/c)


Saving turtle (1).jpeg to turtle (1).jpeg
Saving turtle (1).jpg to turtle (1).jpg
Saving turtle (1).png to turtle (1).png
Saving turtle (2).jpeg to turtle (2).jpeg
Saving turtle (2).jpg to turtle (2).jpg
Saving turtle (2).png to turtle (2).png
Saving turtle (3).jpg to turtle (3).jpg
Saving turtle (4).jpg to turtle (4).jpg
Saving turtle (5).jpg to turtle (5).jpg
Saving turtle (6).jpg to turtle (6).jpg
Saving turtle (7).jpg to turtle (7).jpg
Saving turtle (8).jpg to turtle (8).jpg
Saving turtle (9).jpg to turtle (9).jpg
Saving turtle (10).jpg to turtle (10).jpg
Saving turtle (11).jpg to turtle (11).jpg
Saving turtle (12).jpg to turtle (12).jpg
Saving turtle (14).jpg to turtle (14).jpg
Saving turtle (15).jpg to turtle (15).jpg
Saving turtle (16).jpg to turtle (16).jpg
Saving turtle (17).jpg to turtle (17).jpg
Saving turtle (18).jpg to turtle (18).jpg
Saving turtle (19).jpg to turtle (19).jpg
Saving turtle (20).jpg to turtle (20).jpg
Saving turtle (21).jpg to turtle (21).jpg
Saving turtle 


#test accuracy for different objects
coral reef- 72%

shark- 48%

diver - 50%

fish - 79.4%

jellyfish- 88.09

plant-91%

robot- 84.84%

ruins-41.37%

starfish - 97.9

turtle- 98.6%

overall accuracy=75.12


In [81]:
#dictionary for rgb values

color_code = {
    'light blue':[64,156,174],
    'blue':[28,118,150],
    'dark blue': [27,56,90],
    'green': [95,136,72]

}

In [29]:
#TO FIND THE EUCLIDIAN DISTANCE BETWEEN THE NEAREST COLOUR

import numpy as np
def dist(cord):#coordinates
  min=70000000
  index=''
  for key,val in color_code.items():
    distance=np.linalg.norm(val-cord)
    if distance<min:
      min=distance
      index=key

  return index



In [79]:
#TO FIND THE ACCURACY OF COLOUR DETECTION

def acc():
  path_blue='drive/My Drive/Colab Notebooks/test_col/blue/'
  path_darkb='drive/My Drive/Colab Notebooks/test_col/dark blue/'
  path_lightb='drive/My Drive/Colab Notebooks/test_col/light blue/'
  path_green='drive/My Drive/Colab Notebooks/test_col/green/'
  tot=0 #total
  cor=0 #correct
  green=glob.glob(path_green+'*')
  blue= glob.glob(path_blue+'*')
  darkb= glob.glob(path_darkb+'*')
  lightb= glob.glob(path_lightb+'*')

  for i in range(len(green)):
    tot=tot+1
    dc = DominantColors(green[i], clusters) 
    colors = dc.dominantColors()  
    if dist(colors)=='green':
      cor=cor+1
  
  for i in range(len(blue)):
    tot= tot + 1
    dc = DominantColors(blue[i], clusters) 
    colors = dc.dominantColors()  
    #print(dist(colors))
    #print(blue[i])
    if dist(colors)=='blue':
      cor=cor +1
    
  for i in range(len(darkb)):
    tot=tot + 1
    dc = DominantColors(darkb[i], clusters) 
    colors = dc.dominantColors()  
    #print(dist(colors))
    if dist(colors)=='dark blue':
      cor=cor + 1
  
  for i in range(len(lightb)):
    tot= tot + 1
    dc = DominantColors(lightb[i], clusters) 
    colors = dc.dominantColors()  
    #print(dist(colors))
    if dist(colors)=='light blue':
      cor= cor + 1
  print(cor*100/tot)

In [80]:
#USING KMEANS TO DETECT COLOUR

import cv2
from sklearn.cluster import KMeans
from keras.preprocessing import image
import glob

class DominantColors:

    CLUSTERS = None
    IMAGE = None
    COLORS = None
    LABELS = None
    
    
    def __init__(self, image, clusters=3):
        self.CLUSTERS = clusters
        self.IMAGE = image
        
    def dominantColors(self):
    
        #read image
        img = cv2.imread(self.IMAGE)
        
        #convert to rgb from bgr
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                
        #reshaping to a list of pixels
        img = img.reshape((img.shape[0] * img.shape[1], 3))
        
        #save image after operations
        self.IMAGE = img
        
        #using k-means to cluster pixels
        kmeans = KMeans(n_clusters = self.CLUSTERS)
        kmeans.fit(img)
        
        #the cluster centers are our dominant colors.
        self.COLORS = kmeans.cluster_centers_
        
        #save labels
        self.LABELS = kmeans.labels_
        
        #returning after converting to integer from float
        return self.COLORS.astype(int)

#import glob
#path=''
#img = glob.glob(path+'*.jp#g')
#clusters = 1
#img='blue (1).jpeg'
#dc = DominantColors(img, clusters) 
#colors = dc.dominantColors()  
#print(dist(colors))
#print(colors)   
acc()


drive/My Drive/Colab Notebooks/test_col/green/12.jpg
drive/My Drive/Colab Notebooks/test_col/green/images98.jpg
drive/My Drive/Colab Notebooks/test_col/green/ROBOT (158).jpg
drive/My Drive/Colab Notebooks/test_col/green/ruin (227).jpg
83.65384615384616


TEXTURE(COLOUR) ACCURACY  =  83.65%

OBJECT ACCURACY  =  75.12%